In [9]:
# Import module
import pandas as pd
import numpy as np

In [10]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#uploaded = files.upload()

In [11]:
df = pd.read_csv('fraudTest.csv',on_bad_lines='skip')
df.head()

<ipython-input-11-703b672e9b46>:1: DtypeWarning: Columns (2,5,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('fraudTest.csv',on_bad_lines='skip')


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497.0,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1.371817e+09,33.986391,-81.200714,0.0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.436,302.0,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1.371817e+09,39.450498,-109.960431,0.0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496.0,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1.371817e+09,40.495810,-74.196111,0.0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767.0,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1.371817e+09,28.812398,-80.883061,0.0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.017,1126.0,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1.371817e+09,44.959148,-85.884734,0.0


In [12]:
df['is_fraud'].value_counts()

,count
is_fraud,
0.0,81190
1.0,339


In [22]:

columns_to_drop = ['trans_date_trans_time','city_pop','unix_time','Unnamed: 0', 'merchant',
                   'first', 'last', 'street', 'trans_num', 'dob']
df_cleaned_new = df.drop(columns=columns_to_drop, axis=1)
df_cleaned_new.head()


,cc_num,category,amt,gender,city,state,zip,lat,long,job,merch_lat,merch_long,is_fraud
0,2291163933867244,personal_care,2.86,M,Columbia,SC,29209,33.9659,-80.9355,Mechanical engineer,33.986391,-81.200714,0.0
1,3573030041201292,personal_care,29.84,F,Altonah,UT,84002,40.3207,-110.436,"Sales professional, IT",39.450498,-109.960431,0.0
2,3598215285024754,health_fitness,41.28,F,Bellmore,NY,11710,40.6729,-73.5365,"Librarian, public",40.495810,-74.196111,0.0
3,3591919803438423,misc_pos,60.05,M,Titusville,FL,32780,28.5697,-80.8191,Set designer,28.812398,-80.883061,0.0
4,3526826139003047,travel,3.19,M,Falmouth,MI,49632,44.2529,-85.017,Furniture designer,44.959148,-85.884734,0.0


In [23]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
non_numeric_columns = ['category', 'gender', 'job', 'city', 'state']
label_encoders = {}
for col in non_numeric_columns:
    le = LabelEncoder()
    df_cleaned_new[col] = le.fit_transform(df_cleaned_new[col].astype(str))
    df_cleaned_new.head()

In [24]:
df_na_drop = df_cleaned_new.dropna()
df_na_drop

,cc_num,category,amt,gender,city,state,zip,lat,long,job,merch_lat,merch_long,is_fraud
0,2291163933867244,13,2.86,4,159,43,29209,33.9659,-80.9355,280,33.986391,-81.200714,0.0
1,3573030041201292,13,29.84,3,18,47,84002,40.3207,-110.436,396,39.450498,-109.960431,0.0
2,3598215285024754,7,41.28,3,63,37,11710,40.6729,-73.5365,264,40.495810,-74.196111,0.0
3,3591919803438423,11,60.05,4,757,12,32780,28.5697,-80.8191,411,28.812398,-80.883061,0.0
4,3526826139003047,16,3.19,4,248,25,49632,44.2529,-85.017,201,44.959148,-85.884734,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81533,3500969075198072.0,9,12.34,4,747,50,99033.0,47.2271,-117.0819,91,46.261061,-116.308003,0.0
81534,3512365128314616.0,10,257.39,4,31,27,63010.0,38.4305,-90.387,254,37.431114,-89.616074,0.0
81535,3531129874770000.0,14,3.14,3,89,49,5733.0,43.8065,-73.0882,402,44.280838,-72.827635,0.0
81536,3547560454237873.0,8,60.64,4,27,25,49613.0,44.5232,-86.2061,124,44.738534,-85.463196,0.0


In [25]:
X = df_na_drop.drop(['is_fraud'],axis=1)
y = df_na_drop['is_fraud']


In [27]:

scaler = StandardScaler()
scaled = scaler.fit_transform(X)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(scaled, y,test_size=0.2, random_state=42)


In [29]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

65223 65223
16306 16306


In [30]:
model = GaussianNB()

In [31]:
model.fit(X_train, y_train)

GaussianNB()

In [32]:
pred_test = model.predict(X_test)

In [33]:
matrix_test = confusion_matrix(y_test, pred_test)
print(matrix_test)

[[16127   116]
 [   39    24]]


In [34]:
print("Accuracy:", accuracy_score(y_test, pred_test))
print("Classification Report:\n", classification_report(y_test, pred_test))

Accuracy: 0.9904942965779467
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.99      1.00     16243
         1.0       0.17      0.38      0.24        63

    accuracy                           0.99     16306
   macro avg       0.58      0.69      0.62     16306
weighted avg       0.99      0.99      0.99     16306



In [ ]:
from sklearn.naive_bayes import BernoulliNB
new_model = BernoulliNB(class_prior=[0.80, 0.20])

In [ ]:
new_model.fit(X_train,y_train)

BernoulliNB(class_prior=[0.8, 0.2])

In [ ]:

new_pred_test = new_model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, new_pred_test))
print("Classification Report:\n", classification_report(y_test, new_pred_test))

Accuracy: 0.9911736126106673
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00    110718
           1       0.03      0.04      0.03       426

    accuracy                           0.99    111144
   macro avg       0.51      0.52      0.51    111144
weighted avg       0.99      0.99      0.99    111144



In [3]:
from sklearn.cluster import KMeans
inertia = []
k = list(range(1, 11))


In [35]:

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit()
    inertia.append(k_model.inertia_)


TypeError: KMeans.fit() missing 1 required positional argument: 'X'

In [ ]:
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.head()

,k,inertia
0,1,7.224347e+06
1,2,6.171891e+06
2,3,5.533032e+06
3,4,5.140468e+06
4,5,4.836733e+06


In [ ]:
!pip install hvplot
import hvplot.pandas
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 3.6 MB/s eta 0:00:00


:Curve   [k]   (inertia)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
x_pca = pca.fit_transform(scaled)
x_pca[:5]

array([[ 1.27032341, -1.10284733,  0.03536145],
       [-2.10112019,  1.04231593, -1.90051408],
       [ 2.33876731,  0.35747576, -0.77084209],
       [ 0.57729072, -3.1185963 ,  0.2349132 ],
       [ 0.66605351,  1.51421661,  0.80062112]])

In [ ]:
pca.explained_variance_ratio_

array([0.22567946, 0.15980515, 0.0851455 ])

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit_transform(scaled)

array([[3.22912896, 3.17281144, 4.88109173],
       [4.93730162, 4.39939857, 3.35303026],
       [2.63961563, 3.9316599 , 5.07167734],
       ...,
       [5.71749124, 5.67850183, 3.43772249],
       [5.4948022 , 5.2194575 , 3.33299259],
       [3.86491493, 3.09738849, 3.41735521]])

In [ ]:
predict_clusters = kmeans.predict(scaled)
copy = df_cleaned_new.copy()
copy['cluster'] = predict_clusters
copy.head()


,trans_date_trans_time,cc_num,category,amt,gender,city,state,zip,lat,long,job,merch_lat,merch_long,is_fraud,cluster
0,1592741665,2291163933867244,10,2.86,1,157,39,29209,33.9659,-80.9355,275,33.986391,-81.200714,0,1
1,1592741673,3573030041201292,10,29.84,0,16,43,84002,40.3207,-110.4360,392,39.450498,-109.960431,0,2
2,1592741693,3598215285024754,5,41.28,0,61,33,11710,40.6729,-73.5365,259,40.495810,-74.196111,0,0
3,1592741715,3591919803438423,9,60.05,1,764,8,32780,28.5697,-80.8191,407,28.812398,-80.883061,0,1
4,1592741717,3526826139003047,13,3.19,1,247,21,49632,44.2529,-85.0170,196,44.959148,-85.884734,0,0


In [ ]:
copy.hvplot.scatter(
    x="	trans_date_trans_time",
    y="is_fraud",
    c="cluster",
    title="pca clusters"
)

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['\ttrans_date_trans_time']

PandasInterface expects tabular data, for more information on supported datatypes see https://holoviews.org/user_guide/Tabular_Datasets.html

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
numeric_data = copy.select_dtypes(include=['number'])
sns.heatmap(numeric_data.corr(), cmap='coolwarm', annot=True, fmt='.2f', linewidths=0.5)